# Fetching Data from JSON

In [1]:
%pip install --q jq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_community.document_loaders import JSONLoader

In [3]:
import json
from pathlib import Path
from pprint import pprint


loader = JSONLoader(
    file_path='output_lotr_all.json',
    jq_schema='.',
    text_content=False)

data = loader.load()

if data:
    print("filen er fundet")
else: 
    ("kan ikke finde filen")

filen er fundet


In [4]:
#data[0].page_content

# Vector Embeddings

In [5]:
%pip --q install ollama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!ollama pull nomic-embed-text

In [7]:
!ollama list

NAME                       ID              SIZE      MODIFIED               
nomic-embed-text:latest    0a109f422b47    274 MB    Less than a second ago    
phi3:latest                4f2222927938    2.2 GB    2 weeks ago               
gemma2:latest              ff02c3702f32    5.4 GB    2 weeks ago               
llama3:latest              365c0bd3c000    4.7 GB    2 weeks ago               
llama3.2:latest            a80c4f17acd5    2.0 GB    8 weeks ago               
mistral:latest             f974a74358d6    4.1 GB    8 weeks ago               


In [8]:
%pip install protobuf==4.21.0

  Obtaining dependency information for protobuf==4.21.0 from https://files.pythonhosted.org/packages/c5/54/23262ba768d9038d14b84679f9c2c51e93f7f47df181e64e4ffcb12ba7da/protobuf-4.21.0-cp310-abi3-win_amd64.whl.metadata
  Using cached protobuf-4.21.0-cp310-abi3-win_amd64.whl.metadata (62 bytes)
Using cached protobuf-4.21.0-cp310-abi3-win_amd64.whl (524 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
Note: you may need to restart the kernel to use updated packages.


Reason for being yanked: Required python version not configured correctly (https://github.com/protocolbuffers/protobuf/issues/10076)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.21.0 which is incompatible.
google-cloud-vision 3.8.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 4.21.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%pip install --q chromadb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
%pip install --q langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
%pip install --q langchain-ollama 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
#from langchain_community.embeddings import OllamaEmbeddings


In [13]:
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [14]:
#Split and chunck
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=512,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_documents(data)

In [15]:
print(chunks[12])


page_content='his \nMiddle-earth\n \nlegendarium\n, written by someone outside the Tolkien family. This literature includes reference works, atlases, thorough examinations, essay collections, biographical, art books, and recipe books. This category excludes, however, literature directly concerning \nadaptations\n of Tolkien\'s works.\n\nApart from numerous, singular books by various professors, linguists, Medieval scholars, et cetera, secondary literature also includes many compilative series, such as \nTolkien' metadata={'source': 'C:\\Users\\sirah\\Documents\\UNI\\Pony\\Lokal chatbot\\ollama_pdf_rag-main\\ollama_pdf_rag-main\\output_lotr_all.json', 'seq_num': 1}


In [16]:
#Add to vector database
try:
    vector_db.delete_collection()
    print("Vector database deleted successfully")
except: 
    print("No database") 
    
vector_db = Chroma.from_documents(
    documents = chunks,
    embedding = OllamaEmbeddings(model="nomic-embed-text"),
    collection_name = "local-rag"
)

No database


In [17]:
if vector_db:
    print("Database is initialized")

Database is initialized


# Retrieval

In [18]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [19]:
#llm fra ollama
local_model = "llama3.2"
llm = ChatOllama(model=local_model)

In [20]:
prompt_to_csv = """You are a *Lord of the Rings* expert, specialized in delivering precise answers based on the given knowledge. 
                Your task is to generate five alternative formulations of the user's question to help retrieve relevant information from a vector database. 
                The goal is to maximize coverage and specificity while remaining aligned with the original query's intent. 
                Provide these reformulated questions separated by newlines.

                Once the results are retrieved, provide the most concise and accurate answer possible, 
                ensuring it directly addresses the user's query without unnecessary details.

                Original question: {question}"""
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template=prompt_to_csv,
)

In [21]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [22]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
#for x in range(5):
#    result  = chain.invoke("Who are top leaders in different important stats?")
#    print(f"Run {x + 1}: {result} \n" )

In [24]:
#chain.invoke("Who is Aragorn?")

In [25]:
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
%pip install rouge_score

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
%pip install bert_score

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import csv
from datetime import datetime
import evaluate
from evaluate import load

# Define the questions
questions = [
    "Is Sauron the same race as Gandalf?",
    "How old is treebeard?",
    "What was the purpose of the Istari (Wizards) in Middle-earth?",
    "What is Númenor, and why did it fall?",
    "Who are the Valar, and what is their role in Middle-earth?",
    "What is the significance of the Silmarils?",
    "What is the origin of Orcs?",
    "Why was Aragorn known by so many names?",
    "What is the significance of Smaug in Middle-earth’s history?",
    "What is the Undying Lands, and why can only certain beings go there?",
]

answers = [
    "Sauron and Gandalf were both Maiar: a supernatural race that serve the Valar, and many Maiar assumed different forms. Gandalf is one of five Wizards, a group of Maiar originally sent to Middle-earth by the Valar to aid the resistance against Sauron, and were given the appearance of old men.",
    "Gandalf calls Treebeard, the oldest living thing that still walks beneath the Sun upon this Middle-earth. Later, Celeborn would address him as Eldest.",
    "The Istari were sent by the Valar in the Third Age to assist the Free Peoples against Sauron. They were forbidden from using their full powers to dominate or confront him directly, instead guiding and inspiring others. Gandalf, Saruman, Radagast, and the two Blue Wizards fulfilled varying degrees of this mission, with Gandalf ultimately playing the pivotal role in Sauron’s defeat.",
    "Númenor was a great island kingdom gifted to Men by the Valar after the defeat of Morgoth. It became a center of culture and power during the Second Age. However, its people grew arrogant, defied the Valar, and sought immortality. Sauron's corruption of the Númenóreans led to their attempt to invade Valinor, resulting in the island's destruction and the eventual exile of the Faithful to Middle-earth.",
    "The Valar are powerful beings, akin to gods, who shaped and governed Arda (the world). They were created by Eru Ilúvatar and represent different aspects of the world, such as nature, craftsmanship, and healing. While they rarely intervene directly in Middle-earth after the First Age, their guidance and decisions have a profound impact on its history.",
    "The Silmarils are three legendary jewels crafted by Fëanor, containing the light of the Two Trees of Valinor. They symbolize the greatest beauty and the most destructive greed in Tolkien's world. Their theft by Morgoth and the resulting Oath of Fëanor led to the First Age's tragedies, including wars, betrayals, and the eventual downfall of many Elves and Men.",
    "Orcs were originally Elves, captured and corrupted by Morgoth during the early days of Middle-earth. They were twisted through torture and dark sorcery, becoming his servants. This perversion of life was one of Morgoth’s most heinous acts, symbolizing his contempt for creation and his inability to create life on his own.",
    "Aragorn’s many names reflect his complex identity and the roles he played. Strider was his ranger alias, Thorongil was his name during his service in Rohan and Gondor, and Elessar was his kingly title. These names illustrate his journey from a wandering protector to the unifying King of Gondor and Arnor.",
    "Smaug was the last great dragon of Middle-earth, whose greed and destruction exemplified the corrupting power of wealth. His conquest of Erebor disrupted the balance of power in the north, creating a chain of events that ultimately led to the Battle of the Five Armies and the restoration of the Kingdom under the Mountain.",
    "The Undying Lands, or Valinor, are the dwelling place of the Valar and Elves. It is a realm of immortality and peace, separate from the mortal world. Only immortals or those granted special permission (like Frodo and Bilbo) may enter, as it is meant to be a haven for those beyond the toil of mortal life.",
]

# File name for the CSV
csv_file = "lotr_ALL_1_video.csv"

# Open a CSV file for writing
with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Question", "Run Number", "Answer", "Duration (Seconds)", "BLEU-score", "ROUGE-score", "BERT-score", "BERT-precision", "BERT-recall", "BERT-f1", "CORRECT?", "COMPREHENSIVE?", "CONTRADICTORY", "FALSE INFORMATION", "PARTIAL CORRECT"])  # Header row

    # Loop through each question and get answers
    for question, correct_answer in zip(questions, answers):
        for run_number in range(1, 1):  # Adjust the range for the desired number of runs
            start_time = datetime.now()

            try:
                # Simulate chain invocation (replace with actual invocation logic)
                result = chain.invoke(question)
                end_time = datetime.now()
                duration = (end_time - start_time).total_seconds()  # Calculate the duration in seconds
                formatted_duration = f"{duration:.2f}"

                # Evaluate scores
                bleu = evaluate.load("bleu")
                rouge = evaluate.load("rouge")
                bertscore = load("bertscore")

                bleu_results = bleu.compute(predictions=[result], references=[[correct_answer]])
                rouge_results = rouge.compute(predictions=[result], references=[[correct_answer]])
                bert_results = bertscore.compute(predictions=[result], references=[[correct_answer]], model_type="distilbert-base-uncased")
                bert_precision = sum(bert_results["precision"]) / len(bert_results["precision"])
                bert_recall = sum(bert_results["recall"]) / len(bert_results["recall"])
                bert_f1 = sum(bert_results["f1"]) / len(bert_results["f1"])

                # Write results to the CSV
                writer.writerow([question, run_number, result, formatted_duration, bleu_results, rouge_results, bert_results, bert_precision, bert_recall, bert_f1])
                print(f"Run {run_number} for question '{question}': {result} \nTime taken: {formatted_duration} seconds {bleu_results} {rouge_results} {bert_results} {bert_precision}\n")

            except Exception as e:
                end_time = datetime.now()
                duration = (end_time - start_time).total_seconds()  # Calculate the duration in seconds
                formatted_duration = f"{duration:.2f}"

                # Log error in CSV
                print(f"Error on run {run_number} for question '{question}': {e}")
                writer.writerow([question, run_number, "Error occurred", formatted_duration])

        # Add five extra newline rows between each question
        writer.writerow([""])

# Calculate the average precision, recall, and F1 scores for each question
question_metrics = {}

with open(csv_file, mode="r", encoding="utf-8") as file:
    reader = csv.DictReader(file)

    for row in reader:
        # Skip rows without valid metrics
        question = row["Question"]
        bert_precision = row.get("BERT-precision")
        bert_recall = row.get("BERT-recall")
        bert_f1 = row.get("BERT-f1")

        if question:
            if question not in question_metrics:
                question_metrics[question] = {"precision": [], "recall": [], "f1": []}
            try:
                if bert_precision:
                    question_metrics[question]["precision"].append(float(bert_precision))
                if bert_recall:
                    question_metrics[question]["recall"].append(float(bert_recall))
                if bert_f1:
                    question_metrics[question]["f1"].append(float(bert_f1))
            except ValueError:
                continue

average_metrics = {
    q: {
        "precision": sum(scores["precision"]) / len(scores["precision"]) if scores["precision"] else 0,
        "recall": sum(scores["recall"]) / len(scores["recall"]) if scores["recall"] else 0,
        "f1": sum(scores["f1"]) / len(scores["f1"]) if scores["f1"] else 0,
    }
    for q, scores in question_metrics.items()
}

# Append the averages to the CSV file
with open(csv_file, mode="a", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow([])  # Add a blank line
    writer.writerow(["Question", "Average BERT-precision", "Average BERT-recall", "Average BERT-f1"])  # Header for averages

    for question, metrics in average_metrics.items():
        writer.writerow([question, f"{metrics['precision']:.4f}", f"{metrics['recall']:.4f}", f"{metrics['f1']:.4f}"])
        print(f"Average metrics for '{question}': Precision={metrics['precision']:.4f}, Recall={metrics['recall']:.4f}, F1={metrics['f1']:.4f}")


# FLASK

In [29]:
%pip install flask

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
%pip install flask-cors

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
# Flask API
from flask import Flask, request, jsonify
import threading
from flask_cors import CORS


# Create a fresh Flask app instance
app = Flask(__name__)
CORS(app)
# Define the function to be exposed
def chain_invoke(question):
    # This would call your `chain.invoke` method in the notebook
    # Example placeholder logic:
    response = chain.invoke(question)  # Assuming chain is defined in the notebook
    return response
# Root route
@app.route('/')
def home():
    return "Flask API is running! Use the /invoke endpoint for queries."
# Define API endpoint
@app.route('/invoke', methods=['POST'])
def invoke():
    try:
        # Parse JSON request
        data = request.json
        question = data.get('question', '')

        if not question:
            return jsonify({"error": "Question is required"}), 400

        # Call your chain.invoke function
        response = chain.invoke(question)

        # Return the response to the client
        return jsonify({"response": response})
    except Exception as e:
        return jsonify({"error": str(e)}), 500
# Run Flask app in a background thread
def run_flask():
    app.run(port=5000, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()
print("Flask app is running in the background on port 5000.")

Flask app is running in the background on port 5000.
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Jan/2025 09:19:07] "OPTIONS /invoke HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2025 09:19:17] "POST /invoke HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2025 09:21:51] "OPTIONS /invoke HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2025 09:21:59] "POST /invoke HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2025 09:23:31] "OPTIONS /invoke HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2025 09:23:38] "POST /invoke HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2025 09:24:13] "OPTIONS /invoke HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2025 09:24:22] "POST /invoke HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2025 09:26:44] "OPTIONS /invoke HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2025 09:26:51] "POST /invoke HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2025 09:27:20] "OPTIONS /invoke HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2025 09:27:26] "POST /invoke HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2025 09:27:51] "OPTIONS /invoke HTTP/1.1" 200 -
127.0.0.1 - - [05/Jan/2025 09:28:00] "POST /invoke 